In [1]:
import scipy.io
import os
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
import Butterworth_filtering

from BW_metric import BW_dist
from AI_metric import AI_dist
from manifold_project_toolbox import F_dist
from manifold_project_toolbox import bw_projection_mean
from manifold_project_toolbox import ai_projection_mean
from manifold_project_toolbox import x2corr

from load_BCI_III_IIIa import import_data

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # enable multiple output in one cell

In [2]:
def MDM_dist(A, B, method):
    if method == "BW":
        output = BW_dist(A,B)
    elif method == "AI":
        output = AI_dist(A,B)
    elif method == "Euc":
        output = F_dist(A,B)
    return output

def MDM_mean(x, eps, method, verbose=False):
    # input: psd matrices [N,n,n]
    # output: mean matrix [n,n]
    if method == "BW":
        output = bw_projection_mean(x, eps, verbose=verbose)
    elif method == "AI":
        output = ai_projection_mean(x, eps, verbose=verbose)
    elif method == "Euc":
        output = np.mean(x, axis=0) # arithmetic mean
    return output

In [3]:
# BCI III IIIa
subject = "k3b"
data_train, data_test, label_train, label_test = import_data(subject)
data_train_filtered = Butterworth_filtering.filter_all(data_train, order=5, lowcut=8, highcut=30, fs=250)

# left hand (class 1) vs. right hand (class 2)
idx = np.where(np.isin(label_train, [1, 2]))[0]
sample = x2corr(data_train_filtered)
sample = sample[idx]
label = label_train[idx]

sample.shape
label.shape

(60, 60, 60)

(60,)

In [4]:
# parameter setting
method = "AI"
repetition = 10 # number of reps for cv
k = 6 # k-fold cv

accuracy = np.zeros((repetition, k), dtype = np.float32)
execution_time = np.zeros((repetition, k), dtype = np.float32)

for rep_idx in range(repetition):

    ##### k-fold split
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=rep_idx+22)
    x_train_kfold = []
    y_train_kfold = []
    x_test_kfold = []
    y_test_kfold = []
    for train_idx, test_idx in kfold.split(sample, label):
        x_train_kfold.append(sample[train_idx])
        y_train_kfold.append(label[train_idx])
        x_test_kfold.append(sample[test_idx])
        y_test_kfold.append(label[test_idx])

    ##### MDM
    for fold_idx in range(k):
        # choose fold
        x_train = x_train_kfold[fold_idx]
        y_train = y_train_kfold[fold_idx]
        x_test = x_test_kfold[fold_idx]
        y_test = y_test_kfold[fold_idx]
        # compute mean of each class
        start_time = time.time() # timer starts
        mean1 = MDM_mean(x_train[np.where(y_train==1)], 0.01, method, verbose=False)
        mean2 = MDM_mean(x_train[np.where(y_train==2)], 0.01, method, verbose=False)
        # assign each testing sample to the nearest class mean
        y_pred = np.zeros(x_test.shape[0], dtype = np.float32)
        for test_idx in range(x_test.shape[0]):
            dist1 = MDM_dist(mean1, x_test[test_idx], method)
            dist2 = MDM_dist(mean2, x_test[test_idx], method)
            y_pred[test_idx] = 1 if dist1<dist2 else 2
        end_time = time.time() # timer stops
        # record accuracy and execution time
        accuracy[rep_idx,fold_idx] = sum(y_pred==y_test)/len(y_test)
        execution_time[rep_idx,fold_idx] = end_time - start_time
        
    #     print("Fold " + str(fold_idx+1) + " completed.")
    # print("Repetition " + str(rep_idx+1) + " completed.")

logm result may be inaccurate, approximate err = 6.480342016159539e-06
logm result may be inaccurate, approximate err = 4.298483883521121e-06
logm result may be inaccurate, approximate err = 2.36154978878517e-06
logm result may be inaccurate, approximate err = 5.091567537279926e-06
logm result may be inaccurate, approximate err = 2.8827561665992756e-06
logm result may be inaccurate, approximate err = 4.49652543703918e-06
logm result may be inaccurate, approximate err = 2.5686680613042023e-06
logm result may be inaccurate, approximate err = 2.9708156762102252e-06
logm result may be inaccurate, approximate err = 4.976233094229082e-06
logm result may be inaccurate, approximate err = 1.3215976276272935e-06
logm result may be inaccurate, approximate err = 2.907915220212795e-06
logm result may be inaccurate, approximate err = 4.3985086448682925e-06
logm result may be inaccurate, approximate err = 3.0339262489941594e-06
logm result may be inaccurate, approximate err = 2.942029449303462e-06
lo

In [5]:
np.mean(accuracy)
np.mean(execution_time)

0.68666667

1.5983887

In [6]:
rows = ['Rep{}'.format(i+1) for i in list(range(repetition))]
cols = ['Fold{}'.format(i+1) for i in list(range(k))]

accuracy_df = pd.DataFrame(accuracy, index=rows, columns=cols)
execution_time_df = pd.DataFrame(execution_time, index=rows, columns=cols)

In [7]:
# save_dir = "D:/Projects/BW Revision/Results/BCI III IIIa/"
# # accuracy_df.to_csv(save_dir+subject+"_accuracy_"+method+".csv")
# execution_time_df.to_csv(save_dir+"Execution Time/"+subject+"_execution_time_"+method+".csv")